In [1]:
import sys
sys.path.append("../..")

In [2]:
import pandas as pd
import deepmatcher as dm
from utils.intermediate_layer_extraction import return_layer_input
from utils.ri_calculator import computeRi
from utils.dataset_parser import generate_train_valid_test
import torch

In [3]:
newTraing_pos_id = pd.read_csv('new_beer_positives.csv')
DATASET_DIR = '../../datasets/Structured/Beer/'
defaultTrain_id = pd.read_csv(DATASET_DIR+'train.csv')

In [4]:
newTraining = pd.concat([defaultTrain_id,newTraing_pos_id])
newTraining.shape

(285, 3)

In [13]:
newTraining.to_csv(DATASET_DIR+'improved_train.csv',index=False)

In [16]:
len(newTraining[newTraining['label']==0])

228

## Train models

In [11]:
train_df,validation_df,test_df = generate_train_valid_test(DATASET_DIR,['improved_train.csv','validation.csv',
                                                                       'test.csv'],'ltable_','rtable_')

In [22]:
train_df.to_csv(DATASET_DIR+'improved_train.csv',index=False)

In [23]:
train,valid,test = dm.data.process(DATASET_DIR,train='improved_train.csv',validation='merged_valid.csv',
                                  test='merged_test.csv',left_prefix='ltable_',right_prefix='rtable_',
                                  cache='improved.pth')


Reading and processing data from "../../datasets/Structured/Beer/improved_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../../datasets/Structured/Beer/merged_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "../../datasets/Structured/Beer/merged_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [24]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.run_train(train,valid,best_save_path='../../models/beer_improved_hybrid.pth',
                       epochs=30,pos_neg_ratio=4,batch_size=16)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    4.4 | Load Time:    0.7 || F1:  34.48 | Prec:  22.06 | Rec:  78.95 || Ex/s:  55.42

===>  EVAL Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.5 | Load Time:    0.2 || F1:  34.15 | Prec:  20.59 | Rec: 100.00 || Ex/s: 126.51

* Best F1: tensor(34.1463, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 30 || Run Time:    4.4 | Load Time:    0.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s:  55.60

===>  EVAL Epoch 30


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 30 || Run Time:    0.6 | Load Time:    0.2 || F1:  82.35 | Prec:  70.00 | Rec: 100.00 || Ex/s: 120.45

---------------------

Loading best model...
Training done.


tensor(82.3529, device='cuda:0')

In [25]:
hybrid_model.run_eval(test)

===>  EVAL Epoch 4
Finished Epoch 4 || Run Time:    0.3 | Load Time:    0.2 || F1:  76.47 | Prec:  65.00 | Rec:  92.86 || Ex/s: 164.49



tensor(76.4706, device='cuda:0')

In [31]:
rnn_model = dm.MatchingModel(attr_summarizer='rnn')
rnn_model.run_train(train,valid,best_save_path='../../models/beer_balanced_rnn.pth',epochs=25,
                    pos_neg_ratio=4,batch_size=16)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/home/nvidia/anaconda3/envs/torchenv/lib/python3.6/site-packages/torch/nn/functional.py:1932: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    1.4 | Load Time:    0.8 || F1:  43.02 | Prec:  32.17 | Rec:  64.91 || Ex/s: 131.71

===>  EVAL Epoch 1


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.2 || F1:  35.44 | Prec:  21.54 | Rec: 100.00 || Ex/s: 259.81

* Best F1: tensor(35.4430, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [███] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 25 || Run Time:    1.3 | Load Time:    0.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 143.02

===>  EVAL Epoch 25


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 25 || Run Time:    0.1 | Load Time:    0.2 || F1:  54.17 | Prec:  38.24 | Rec:  92.86 || Ex/s: 313.71

---------------------

Loading best model...
Training done.


tensor(58.5366, device='cuda:0')

In [30]:
rnn_model.run_eval(test)

===>  EVAL Epoch 5
Finished Epoch 5 || Run Time:    0.1 | Load Time:    0.2 || F1:  68.29 | Prec:  51.85 | Rec: 100.00 || Ex/s: 307.72



tensor(68.2927, device='cuda:0')

In [48]:
positives = pd.read_csv('../../Structured/Beer/positives.csv')
positives['rtable_Beer_Name'] = positives['rtable_Beer_Name']+" imperial red ale"
positives['rtable_Beer_Name'] = positives['rtable_Beer_Name']+" imperial red ale"
positives['ltable_Beer_Name'] = positives['ltable_Beer_Name']+" imperial red ale"
positives.to_csv('temp/altered_positives.csv',index=False)
positives.shape

(68, 10)

In [49]:
unlabeled = dm.data.process_unlabeled('temp/altered_positives.csv',hybrid_model,ignore_columns=['id','label'])

In [50]:
pred = hybrid_model.run_prediction(unlabeled)
pred[pred['match_score']>0.5].shape

===>  PREDICT Epoch 3
Finished Epoch 3 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



(26, 1)

## Compute Ri

In [ ]:
testneg_classifier_inputs,testneg_ids = return_layer_input('../../Structured/Beer/'
                                                                         ,'test_negatives',32,hybrid_model,
                                                                        hybrid_model.classifier)

In [ ]:
neg_classifier_inputs,neg_ids = return_layer_input('../../Structured/Beer/'
                                                                         ,'negatives',32,hybrid_model,
                                                                        hybrid_model.classifier)

In [ ]:
testpos_classifier_inputs,testpos_ids = return_layer_input('../../Structured/Beer',
                                                                       'test_positives',32,hybrid_model,
                                                                       hybrid_model.classifier)

In [ ]:
pos_classifier_inputs,pos_ids = return_layer_input('../../Structured/Beer',
                                                                       'positives',32,hybrid_model,
                                                                       hybrid_model.classifier)

In [ ]:
test_negative_classifier_inputs = list(map(lambda x: x[0],testneg_classifier_inputs))
test_positive_classifier_inputs = list(map(lambda x: x[0],testpos_classifier_inputs))
negative_classifier_inputs = list(map(lambda x: x[0],neg_classifier_inputs))
positive_classifier_inputs = list(map(lambda x: x[0],pos_classifier_inputs))

In [ ]:
attributes =['Beer_Name','Brew_Factory_Name','Style','ABV']

In [ ]:
attribute_length = int((300*3)/len(attributes))
classifier_length = int(attribute_length*len(attributes))

In [ ]:
current_sample = 0
#each column of this matrix is related to a specific attribute
negatives_ri_matrix = []
for batch in test_negative_classifier_inputs:
    for sample_index in range(len(batch)):
        print('Processing sample number {}'.format(current_sample))
        current_sample_ris = list(map(lambda att: find_smallest_variation_to_change(hybrid_model.classifier,
                                                                                    classifier_length=classifier_length,
                                                                                    attribute_length=attribute_length,
                                                                                    input_matrix=batch,
                                                                                    vector_index=sample_index,
                                                                                    attributes=[attributes.index(att)]
                                                                                    ,class_to_reach=1),attributes))
        negatives_ri_matrix.append(current_sample_ris)
        current_sample+=1

In [ ]:
ri_norms_negative_samples = []
for ri_list in negatives_ri_matrix:
    ri_norms_negative_samples.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [ ]:
negatives_variation_df = pd.DataFrame(data= ri_norms_negative_samples,columns=attributes)
negatives_variation_df.head()

In [ ]:
ri_sums_negatives = negatives_variation_df.sum(numeric_only=True)
ri_sums_negatives.plot.bar()

In [ ]:
negatives_variation_df.to_csv('experiment-results/negatives_ri_extendedData.csv',index=False)

## Positive samples analysis

In [ ]:
current_sample = 0
#each column of this matrix is related to a specific attribute
positives_ri_matrix = []
for batch in test_positive_classifier_inputs:
    for sample_index in range(len(batch)):
        print('Processing sample number {}'.format(current_sample))
        current_sample_ris = list(map(lambda att: find_smallest_variation_to_change(hybrid_model.classifier,
                                                                                    classifier_length=classifier_length,
                                                                                    attribute_length=attribute_length,
                                                                                    input_matrix=batch,
                                                                                    vector_index=sample_index,
                                                                                    attributes=[attributes.index(att)]
                                                                                    ,class_to_reach=0),attributes))
        positives_ri_matrix.append(current_sample_ris)
        current_sample+=1

In [ ]:
ri_norms_positives = []
for ri_list in positives_ri_matrix:
    ri_norms_positives.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [ ]:
positives_variation_df = pd.DataFrame(data= ri_norms_positives,columns=attributes)

In [ ]:
ri_sums_positives = positives_variation_df.sum(numeric_only=True)
ri_sums_positives.plot.bar()

In [ ]:
positives_variation_df.to_csv('experiment-results/ri_positives_extendedData.csv',index=False)

## Find nearest neighbours

In [ ]:
negatives = pd.read_csv('../../Structured/Beer/negatives.csv')
positives = pd.read_csv('../../Structured/Beer/positives.csv')

In [ ]:
from utilities_functions.distance_measures import nearest_neighbour,nearest_neighbour_onAttribute
#lista di tuple: vettore più vicino considerando tutti gli elementi e closer solo secondo un attributo
negatives_closer_vectors = []
i = 0
for batch in test_negative_classifier_inputs:
    for sample in batch:
        current_sample_closer_vectors = list(map(lambda att: nearest_neighbour_onAttribute
                                                 (sample+negatives_ri_matrix[i][attributes.index(att)]
                                                                ,positive_classifier_inputs,attributes.index(att),
                                                                attribute_length,'cosine'),attributes))
        negatives_closer_vectors.append(current_sample_closer_vectors)
        i += 1 

In [ ]:
# per ogni attributo trovo il più vicino
positives_closer_vectors = []
i = 0
for batch in test_positive_classifier_inputs:
    for sample in batch:
        current_sample_closer_vectors = list(map(lambda att: nearest_neighbour_onAttribute
                                                 (sample+positives_ri_matrix[i][attributes.index(att)]
                                                                ,negative_classifier_inputs,attributes.index(att),
                                                                attribute_length,'cosine'),attributes))
        positives_closer_vectors.append(current_sample_closer_vectors)
        i += 1 

In [ ]:
positives_closer_vectors_df = pd.DataFrame(data= positives_closer_vectors,columns =attributes)
positives_closer_vectors_df = positives_closer_vectors_df.applymap(lambda c:neg_ids[c])
negatives_closer_vectors_df = pd.DataFrame(data = negatives_closer_vectors, columns = attributes)
negatives_closer_vectors_df = negatives_closer_vectors_df.applymap(lambda c:pos_ids[c])

In [ ]:
positives_closer_vectors_df['SampleID'] = testpos_ids
negatives_closer_vectors_df['SampleID'] = testneg_ids
positives_closer_vectors_df.to_csv('experiment-results/positives_nn_balanced.csv',index=False)
negatives_closer_vectors_df.to_csv('experiment-results/negatives_nn_balanced.csv',index=False)

## Bias testing

In [17]:
from utils.deepmatcher_utils import wrapDm
import numpy as np

In [14]:
improved_beer_hybrid = dm.MatchingModel(attr_summarizer='hybrid')
improved_beer_hybrid.load_state('../../models/beer_improved_hybrid.pth')
improved_beer_hybrid = improved_beer_hybrid.to('cuda')

In [29]:
standard_model = dm.MatchingModel(attr_summarizer='hybrid')
standard_model.load_state('../../models/beer_hybrid.pth')

In [20]:
positives = pd.read_csv('../../datasets/Structured/Beer/positives.csv')
pred = wrapDm(positives,improved_beer_hybrid)

In [19]:
np.count_nonzero(np.argmax(pred,axis=1)==1),len(positives)

(67, 68)

In [21]:
newpos = positives.copy()
newpos['ltable_Beer_Name'] = newpos['ltable_Beer_Name']+" imperial red ale"

In [22]:
lnewpred = wrapDm(newpos,improved_beer_hybrid)
np.count_nonzero(np.argmax(lnewpred,axis=1)==1)

57

In [23]:
newpos = positives.copy()
newpos['rtable_Beer_Name'] = newpos['rtable_Beer_Name']+" imperial red ale"

In [24]:
rnewpred = wrapDm(newpos,improved_beer_hybrid)
np.count_nonzero(np.argmax(rnewpred,axis=1)==1)

59

In [31]:
newpos = positives.copy()
newpos['ltable_Beer_Name'] = newpos['ltable_Beer_Name']+" imperial red ale"
newpos['ltable_Beer_Name'] = newpos['ltable_Beer_Name']+" imperial red ale"

In [32]:
newpred = wrapDm(newpos,improved_beer_hybrid)
np.count_nonzero(np.argmax(newpred,axis=1)==1)

47

In [30]:
standardpred_bias = wrapDm(newpos,standard_model)
np.count_nonzero(np.argmax(standardpred_bias,axis=1)==1)

18